# RAG Setup
Now that we have the structured data in ```JSONL``` format, we will go about embedding and vectorizing this data to make it RAG-ready.   
   
So, before any LLM "answers" in a RAG pipeline, the setup must go through:
> Data → Embedding → Vectorization → Indexing → Retrieval → LLM QA

## Load Data

In [1]:
import json

# Path to your JSONL file
jsonl_path = "/workspaces/Avva-2.0/experiments/symptom_remedy_data.jsonl"

# Load records
records = []
with open(jsonl_path, "r") as f:
    for line in f:
        try:
            item = json.loads(line.strip())
            # Optional: filter out invalid entries
            if item.get("symptom") and item.get("remedy"):
                records.append(item)
        except json.JSONDecodeError as e:
            print(f"Skipping invalid line: {e}")


In [2]:
print(f"Loaded {len(records)} valid records.")
print(records[0])

Loaded 88 valid records.
{'symptom': 'cough and sore throat', 'remedy': 'Tea, Honey, Echinacea, Elderberry syrup, Pelargonium', 'description': 'Throat-coating properties to reduce irritation, soothe sore throats, suppress coughs, reduce cold symptoms, and have antiviral properties', 'warnings': 'Try each remedy individually and consult a doctor before use if pregnant, breastfeeding, or taking prescription medications', 'source_url': 'https://www.allinahealth.org/healthysetgo/heal/natural-remedies-for-everyday-illnesses'}


## RAG Framework 

| Framework               | Best For                           | Strengths                                                    | Tradeoffs                                |
| ----------------------- | ---------------------------------- | ------------------------------------------------------------ | ---------------------------------------- |
| **LangChain**           | General RAG + agents               | ✅ Huge ecosystem, tools, agents, integrations                | Heavy; complex tracing; less transparent |
| **LlamaIndex**          | Structured, hybrid, or tabular RAG | ✅ Better document structure control, tree index, tabular RAG | Agents are basic, fewer agent tools      |
| **Haystack**            | Traditional NLP, QA, open source   | ✅ Flexible pipeline control, solid evals                     | Less LLM-native; older ecosystem         |
| **SmolAgents / CrewAI** | Dev-first agent systems            | ✅ Tiny, hackable, very fast for multi-agent apps             | Lacks fine-grained RAG plumbing          |
| **RAGatouille**         | Plug-and-play vector RAG           | ✅ Super minimal RAG-only wrapper                             | No agents, no eval tools                 |
| **Dust.tt**             | UI-first prompt+tool orchestration | ✅ Great UX, graphs, eval tracking                            | SaaS-based, not self-hosted              |      
   
So module-wise
   
| Module                   | Best Choice                                                                                              |
| ------------------------ | -------------------------------------------------------------------------------------------------------- |
| 🔍 Embedding + Retrieval | **LlamaIndex** or **LangChain** (start with LangChain if agents are central)                             |
| 🧠 Agents + Tool Use     | **CrewAI** or **SmolAgents** (lightweight, composable)                                                   |
| 📊 Evaluation            | Use **TruLens**, **RAGAS**, or **Promptfoo** (can be plugged into either)                                |
| 🔧 Optional RAG Plug     | You can even use **LlamaIndex for vector management**, and connect to **LangChain Agents** for reasoning |

### Step 1: Convert JSON → `Document` objects
| Format                      | Pros                                               | Cons                                                    |
| --------------------------- | -------------------------------------------------- | ------------------------------------------------------- |
| `langchain.schema.Document` | ✅ Standardized for LangChain<br>✅ Metadata support | Slightly verbose                                        |
| `llama_index.TextNode`      | ✅ Structured field storage<br>✅ Fine-tuned control | Works best if you plan to use **LlamaIndex end-to-end** |
| Raw dicts (custom)          | ✅ Minimal, DIY logic                               | ❌ Not supported in RAG toolchains                       |
   
✅ Use LangChain


In [3]:
from langchain.schema import Document

documents = []
for item in records:
    if item.get("symptom") and item.get("remedy"):
        page_content = (
            f"Symptom: {item['symptom']}\n"
            f"Remedy: {item['remedy']}\n"
            f"Description: {item.get('description', '')}\n"
            f"Warnings: {item.get('warnings', '')}"
        )
        metadata = {"source_url": item.get("source_url", "unknown")}
        documents.append(Document(page_content=page_content, metadata=metadata))


In [4]:
documents[:5]

[Document(metadata={'source_url': 'https://www.allinahealth.org/healthysetgo/heal/natural-remedies-for-everyday-illnesses'}, page_content='Symptom: cough and sore throat\nRemedy: Tea, Honey, Echinacea, Elderberry syrup, Pelargonium\nDescription: Throat-coating properties to reduce irritation, soothe sore throats, suppress coughs, reduce cold symptoms, and have antiviral properties\nWarnings: Try each remedy individually and consult a doctor before use if pregnant, breastfeeding, or taking prescription medications'),
 Document(metadata={'source_url': 'https://www.allinahealth.org/healthysetgo/heal/natural-remedies-for-everyday-illnesses'}, page_content='Symptom: upset stomach, nausea, motion sickness\nRemedy: Ginger\nDescription: Helpful for digestive issues\nWarnings: None'),
 Document(metadata={'source_url': 'https://www.allinahealth.org/healthysetgo/heal/natural-remedies-for-everyday-illnesses'}, page_content='Symptom: diarrhea\nRemedy: Probiotics\nDescription: Good bacteria for the 

### Step 2: Choose Embedding Model

| Model                                                   | Pros                                                     | Cons                                                                  |
| ------------------------------------------------------- | -------------------------------------------------------- | --------------------------------------------------------------------- |
| `OpenAIEmbeddings` (`text-embedding-3-small`)           | ✅ Very accurate<br>✅ Works out of the box with LangChain | ❌ API calls = paid<br>❌ Slow for large batches                        |
| `HuggingFaceEmbeddings` (e.g. MiniLM, bge-small)        | ✅ Free<br>✅ Fast<br>✅ Good quality                       | ❌ Needs local compute<br>❌ Slightly less performant on subtle queries |
| `InstructorEmbeddings`                                  | ✅ Good with instructions/context<br>✅ Free               | ❌ Heavier; needs sentence-transformers setup                          |
| `Mistral-based` embeddings (via Hugging Face or Ollama) | ✅ Emerging, fast, local                                  | ❌ Less mature; limited support in LangChain now                       |



In [5]:
! pip install sentence-transformers langchain-community chromadb langchain-huggingface --quiet

For general-purpose semantic search, a good starting model is:
`"all-MiniLM-L6-v2"`   

Other great options:   

`"BAAI/bge-small-en"` – very strong performance.   
`"intfloat/multilingual-e5-small"` – if you want multilingual support.   
`"thenlper/gte-small"` – open and tuned for RAG-like tasks.   
   
We'll use `"all-MiniLM-L6-v2"` here:    


In [6]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2"
)

/workspaces/Avva-2.0/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Step 3: Choose Vector Store

| Store                       | Pros                                                            | Cons                                         |
| --------------------------- | --------------------------------------------------------------- | -------------------------------------------- |
| **Chroma**                  | ✅ Lightweight<br>✅ Local file persistence<br>✅ LangChain native | Slightly slower for huge datasets            |
| **FAISS**                   | ✅ Fast, local, simple                                           | ❌ No metadata filtering natively             |
| **Weaviate / Qdrant**       | ✅ Scalable<br>✅ Metadata filtering<br>✅ REST API                | Requires running a server or cloud account   |
| **Pinecone**                | ✅ Hosted, powerful<br>✅ Metadata filtering                      | ❌ Paid API<br>❌ Cold starts in free tier     |
| **LlamaIndex vector index** | ✅ Tight integration with LlamaIndex pipelines                   | ❌ Limited if not using LlamaIndex end-to-end |
   
✅ Start with Chroma — best for local, fast, simple vector DB with metadata support.

In [7]:
from langchain.vectorstores import Chroma

vector_db = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory="avva_rag_vector_db"
)


## Test Basic Retrieval

In [8]:
retriever = vector_db.as_retriever()
results = retriever.invoke("home remedies for cough")

for i, doc in enumerate(results):
    print(f"\n🔹 Result {i+1}")
    print(doc.page_content)
    print(doc.metadata)



🔹 Result 1
Symptom: Cough
Remedy: Apple cider vinegar, cayenne & cinnamon
Description: A mixture of apple cider vinegar, cayenne, cinnamon, and honey can help soothe a cough and sore throat.
Warnings: None
{'source_url': 'https://www.prevention.com/health/a20477585/35-all-time-favorite-natural-remedies/'}

🔹 Result 2
Symptom: cough and sore throat
Remedy: Tea, Honey, Echinacea, Elderberry syrup, Pelargonium
Description: Throat-coating properties to reduce irritation, soothe sore throats, suppress coughs, reduce cold symptoms, and have antiviral properties
Warnings: Try each remedy individually and consult a doctor before use if pregnant, breastfeeding, or taking prescription medications
{'source_url': 'https://www.allinahealth.org/healthysetgo/heal/natural-remedies-for-everyday-illnesses'}

🔹 Result 3
Symptom: Cough
Remedy: Honey
Description: Honey may work just as well for a cough as over-the-counter medicines, especially helpful for children who aren’t old enough to take those.
Warn

### General Knowledge

#### 🧠 Embedding Models: Comparison Table

| Model / Tool                          | Type    | Strengths                                      | Weaknesses                                    | Hosted / Local | LangChain Support | 💸 Cost               |
|--------------------------------------|---------|------------------------------------------------|-----------------------------------------------|----------------|-------------------|------------------------|
| **OpenAI (`text-embedding-3-small`)**| Dense   | High quality, optimized for LLM RAG           | Paid, API limits                              | Hosted         | ✅ Yes            | 💸 $0.00002 / 1K tokens |
| **HuggingFace Transformers**         | Dense   | Free, fast, open-source                       | Slightly lower accuracy than OpenAI           | Local / HF Hub | ✅ Yes            | 🆓 Free (local use)     |
| **Cohere Embed v3**                  | Dense   | Fast, high quality, multilingual              | API-based, requires key                       | Hosted         | ✅ Yes            | 💸 $0.10 / 1K calls     |
| **Jina AI Embeddings**               | Dense   | Open-source, designed for long docs           | Less mainstream                                | Both           | ✅ Yes            | 🆓 Free (OSS), 💸 if hosted |
| **Google's BERT / USE**              | Dense   | Strong sentence-level understanding           | Not search-optimized                          | Local          | 🚫 No             | 🆓 Free                 |
| **BM25 / TF-IDF**                    | Sparse  | Great keyword matching                        | Poor semantic understanding                   | Local          | ✅ Yes            | 🆓 Free                 |
| **Hybrid (Dense + Sparse)**          | Hybrid  | Combines semantic + keyword strengths         | Complex to manage                             | Local          | ✅ Yes            | 🆓 Free                 |

###### ✅ Best by Use Case:

| Use Case                              | Recommended Embedding     |
|--------------------------------------|----------------------------|
| RAG with semantic accuracy           | OpenAI or Cohere           |
| Privacy/local deployment             | HuggingFace, BM25          |
| Long documents (web scraping, RAG)   | Jina AI, OpenAI            |
| Keyword-driven search                | BM25 or Hybrid             |

---

#### 🧊 Vector Database Options: Comparison Table

| Vector DB        | Hosted / Local | Dedup Support | Metadata Filtering | Scale Ready | Notes                              | 💸 Cost                          |
|------------------|----------------|----------------|---------------------|-------------|-------------------------------------|----------------------------------|
| **Chroma**        | Local           | ❌ No           | ✅ Yes               | ⚠️ Limited   | Great for prototyping              | 🆓 Free                          |
| **FAISS**         | Local           | ❌ No           | ❌ No                | ✅ Yes       | No metadata, high perf              | 🆓 Free                          |
| **Pinecone**      | Hosted          | ✅ Yes (ID)     | ✅ Yes               | ✅ Yes       | Ideal for production RAG           | 💸 Free tier + $0.096/1M vec updates |
| **Qdrant**        | Both            | ✅ Yes (ID)     | ✅ Yes               | ✅ Yes       | Open source + managed options      | 🆓 OSS, 💸 $5+/mo (cloud)         |
| **Weaviate**      | Both            | ✅ Yes          | ✅ Yes + hybrid      | ✅ Yes       | REST API + hybrid search           | 🆓 OSS, 💸 cloud available        |
| **Milvus**        | Both            | ✅ Manual       | ✅ Yes               | ✅ Yes       | Best for massive vector data       | 🆓 Free                          |
| **ElasticSearch**| Both            | ⚠️ Partial      | ✅ Yes               | ✅ Yes       | Keyword + dense hybrid search      | 💸 Cloud: starts ~$16/mo         |

###### ✅ Best by Use Case:

| Need                                 | Recommended Store |
|-------------------------------------|-------------------|
| Local quick testing                 | Chroma, FAISS     |
| Deduplicated, production-ready RAG | Pinecone, Qdrant  |
| Open source, scalable               | Qdrant, Milvus    |
| Hybrid search (semantic + keyword) | Weaviate, ElasticSearch |
| Fine-grained filtering              | Qdrant, Weaviate  |

---

#### 🔄 Recommended Pairings

| Embedding        | Best Paired Vector DB     |
|------------------|---------------------------|
| OpenAI           | Pinecone, Qdrant, Weaviate|
| HuggingFace      | Chroma, FAISS, Qdrant     |
| Cohere           | Pinecone, Qdrant          |
| BM25 / Hybrid    | Weaviate, ElasticSearch   |
| Jina AI          | Qdrant, Chroma            |

